In [1]:
import tweepy
import json
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os
import time
import sys
from tweepy import OAuthHandler
from tweepy import Stream, Cursor
from tweepy.streaming import StreamListener
from random import sample

In [2]:
from datetime import datetime, timedelta

In [3]:
path = '../Desktop/AssignmentSMC/'

In [4]:
secrets = json.loads(open('secrets.json').read())
api_key = secrets['CONSUMER_KEY']
api_secret_key = secrets['CONSUMER_SECRET']
access_token = secrets['ACCESS_TOKEN']
access_token_secret = secrets['ACCESS_TOKEN_SECRET']

auth = OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [5]:
screen_name = "hulu" #your brand name

# Followers

In [ ]:
MAX_FRIENDS = 1000000
followers = []
friends = []
followers_info = []
friends_info = []

def paginate(items, n):
    """Generate n-sized chunks from items"""
    for i in range(0, len(items), n):
        yield items[i:i+n]

if __name__ == '__main__':
    screen_name = "hulu" #enter screen name on who you would like to check on it.
    print("collecting data for " + screen_name)
    dirname = "Assignment/users/{}".format(screen_name)
    max_pages = math.ceil(MAX_FRIENDS / 5000)
    try:
        os.makedirs(dirname, mode=0o755, exist_ok=True)
    except OSError:
        print("Directory {} already exists".format(dirname))
    except Exception as e:
        print("Error while creating directory {}".format(dirname))
        print(e)
        sys.exit(1)
    """
    # get user's profile
    fname = "Assignment/users/{}/user_profile.json".format(screen_name)
    with open(fname, 'w') as f:
        profile = api.get_user(screen_name=screen_name)        
        f.write(json.dumps(profile._json, indent=4))
    """
    
    # get followers for a given user
    print("Followers list")
    fname = "Assignment/users/{}/followers.json".format(screen_name)
    with open(fname, 'w') as f:
        for follower in Cursor(api.followers_ids, screen_name=screen_name).pages(max_pages):
            
            for chunk in paginate(follower, 100):
                users = api.lookup_users(user_ids=chunk)
                for user in users:
                    followers_info = {'Name': user.screen_name, 'Followers': user.followers_count,'Friends': user.friends_count,'Created_at': user.created_at,'Location': user.location}
                    followers.append(followers_info)
                    f.write(json.dumps(user._json)+"\n")                 
            if len(follower) == 5000:
                print("More results available. Sleeping for 60 seconds to avoid rate limit")
                time.sleep(60)
                
    with open('hulu_followers.csv', 'a', newline='', encoding='utf-8') as file2:
        writer = csv.DictWriter(file2, fieldnames=['Name', 'Followers', 'Friends','Created_at','Location'])
        writer.writeheader()
        
        for data in followers:
            writer.writerow(data)

    print("task completed")

In [ ]:
MAX_FRIENDS = 1000000
followers = []
friends = []
followers_info = []
friends_info = []

def paginate(items, n):
    """Generate n-sized chunks from items"""
    for i in range(0, len(items), n):
        yield items[i:i+n]

if __name__ == '__main__':
    screen_name = "hulu" #enter screen name on who you would like to check on it.
    print("Collecting data for " + screen_name)
    dirname = "Assignment/users/{}".format(screen_name)
    max_pages = math.ceil(MAX_FRIENDS / 5000)
    try:
        os.makedirs(dirname, mode=0o755, exist_ok=True)
    except OSError:
        print("Directory {} already exists".format(dirname))
    except Exception as e:
        print("Error while creating directory {}".format(dirname))
        print(e)
        sys.exit(1)
    """
    # get user's profile
    fname = "Assignment/users/{}/user_profile.json".format(screen_name)
    with open(fname, 'w') as f:
        profile = api.get_user(screen_name=screen_name)        
        f.write(json.dumps(profile._json, indent=4))
    """

    # get friends for a given user
    print("\n")
    print("Friends list")
    fname = "Assignment/users/{}/friends.json".format(screen_name)
    with open(fname, 'w') as f:
        for friend in Cursor(api.friends_ids, screen_name=screen_name).pages(max_pages):
            for chunk in paginate(friend, 100):
                users = api.lookup_users(user_ids=chunk)
                for user in users:
                    friends_info = {'Name': user.screen_name, 'Followers': user.followers_count,'Friends': user.friends_count, 'Created_at': user.created_at,'Location': user.location}
                    friends.append(friends_info)
                    f.write(json.dumps(user._json)+"\n")
            if len(friend) == 5000:
                print("More results available. Sleeping for 60 seconds to#  avoid rate limit")
                time.sleep(60)
                
    with open('hulu_friends.csv', 'a', newline='', encoding='utf-8') as file2:
        writer = csv.DictWriter(file2, fieldnames=['Name', 'Followers', 'Friends','Created_at','Location'])
        writer.writeheader()
        
        for data in friends:
            writer.writerow(data)
              
    print("Task completed")

# Followers Count

In [15]:
user = api.get_user(screen_name)
fll = user.followers_count
fstr = str(fll)
date = datetime(2021, 2, 8).strftime('%Y-%m-%d')
header = ['Date Collected', 'Count']
rows = [date, fll]

with open('hulu_followers.csv', 'a+') as csv_file:
    write = csv.writer(csv_file)
    #write.writerow(header)
    write.writerow(rows)

# Mentions

In [28]:
date = datetime(2021, 2, 26)
tomorrow = date + timedelta(1)

In [29]:
datetime.strftime(tomorrow, '%Y-%m-%d')

'2021-02-27'

In [30]:
# Using Cursor to get more than 100 tweets at one time
query = '@hulu -filter:retweets'
startDate = datetime.strftime(date, '%Y-%m-%d')
endDate = datetime.strftime(tomorrow, '%Y-%m-%d')
searched_tweets = [status for status in tweepy.Cursor(api.search, q=query, since=startDate, until=endDate).items()]

# JSON
with open('hulu_mentions.json', 'w') as f:
  for tweet in searched_tweets:
    f.write(json.dumps(tweet._json) + '\n')

# # After 14 days, convert json to CSV
# df = pd.read_json (r'iqiyi_mentions.json', lines=True)
# export_csv = df.to_csv (r'converted.csv', index = None, header=True)

df = pd.read_json (r'hulu_mentions.json', lines=True)
export_csv = df.to_csv (r'hulu_mentions.csv', index=None, mode='a', header=False) # For the first time, header = True

In [34]:
df = pd.read_json (r'hulu_mentions_9feb.json', lines=True)
export_csv = df.to_csv (r'hulu_mentions2.csv', index=None, mode='a', header=True) # For the first time, header = True

In [ ]:
# df = pd.read_csv('converted.csv')

In [ ]:
# Convert Object to DateTime (After 14 Days)
# df['created_at'] = pd.to_datetime(df['created_at'])

In [ ]:
# Split Date & Time (After 14 days)
# df['date'] = [d.date() for d in df['created_at']]
# df['time'] = [d.time() for d in df['created_at']]

# Tweets

In [ ]:
# tweetss = api.user_timeline(screen_name, since=startDate, until=endDate)
# i = 1

# for twit in tweetss:
#   if twit.lang == "en":
#     print(i, twit.text, "|", twit.created_at.time())
#     i += 1

# with open('hulu_tweets.json', 'w') as f:
#   for tweet in tweetss:
#     f.write(json.dumps(tweet._json) + '\n')

# df = pd.read_json (r'hulu_tweets.json', lines=True)
# export_csv = df.to_csv (r'hulu_tweets.csv', index = None, mode='a', header=False) # For the first time, header = True

In [9]:
tweetss = [status for status in tweepy.Cursor(api.user_timeline, screen_name=screen_name, since_id=1359109793139617793, until_id=).items()]

with open('hulu_tweets2.json', 'w') as f:
  for tweet in tweetss:
    f.write(json.dumps(tweet._json) + '\n')

df = pd.read_json (r'hulu_tweets2.json', lines=True)
export_csv = df.to_csv (r'hulu_tweets2.csv', index = None, mode='a', header=True) # For the first time, header = True